In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import os



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
BASE_PATH = '../input/gtsrb-german-traffic-sign/'
TRAIN_PATH = BASE_PATH + 'Train'


data = []
labels = []

NUM_CLASSES = len(os.listdir(TRAIN_PATH))
IMG_HEIGHT = 48
IMG_WIDTH = 48
IMG_CHAN = 3

img_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHAN)

NUM_CATEGORIES = len(os.listdir(TRAIN_PATH))
for category in range(NUM_CLASSES):
        categories = os.path.join(TRAIN_PATH, str(category))
        for img in os.listdir(categories):
            image = Image.open(os.path.join(categories, img))
            image = image.resize((IMG_HEIGHT,IMG_WIDTH))
            image = np.array(image)
            image = image/255.0
            data.append(image)


In [ ]:
# Convert lists to NumPy arrays
data = np.array(data)


# create a training and validation split
img_train, img_val = train_test_split(data,test_size=0.2, random_state=16)
print(img_train.shape, img_val.shape)


In [ ]:
ae = tf.keras.models.Sequential()
ae.add(tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', input_shape=img_shape))
ae.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
ae.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
ae.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
ae.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
ae.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
ae.add(tf.keras.layers.Flatten())
ae.add(tf.keras.layers.Dense(1024, activation='relu'))
ae.add(tf.keras.layers.Dense(256, activation='relu'))
ae.add(tf.keras.layers.Dense(1024, activation='relu'))
ae.add(tf.keras.layers.Dense(5184, activation='relu'))
ae.add(tf.keras.layers.Reshape((9, 9, 64)))
ae.add(tf.keras.layers.UpSampling2D(size=(2, 2)))
ae.add(tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), activation='relu'))
ae.add(tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(3, 3), activation='relu'))
ae.add(tf.keras.layers.UpSampling2D(size=(2, 2)))
ae.add(tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=(3,3), activation='relu'))
ae.add(tf.keras.layers.Conv2DTranspose(filters=3, kernel_size = (3,3), activation = 'relu'))
ae.summary()

In [ ]:
ae.compile(loss='MSE', optimizer='adam', metrics=['accuracy'])

history = ae.fit(img_train, img_train, batch_size=64, epochs=40, validation_data=(img_val, img_val))

In [ ]:
ae.save("autoencoder_v1.h5")

In [ ]:
test_data = []
TEST_PATH = BASE_PATH + 'Test/'

y_test = pd.read_csv(BASE_PATH+'Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values


for img in imgs:
    image = Image.open(os.path.join(BASE_PATH+img))
    image = image.resize((IMG_HEIGHT,IMG_WIDTH))
    image = np.array(image)
    image = image/255.0
    test_data.append(image)

test_data = np.array(test_data)            
gen = ae.predict(test_data)


In [ ]:
# Load Chinese Traffic Signs

ADV_PATH = '../input/chinese-ts/images/'

adv_data = []

for img in os.listdir(ADV_PATH):
    image = Image.open(os.path.join(ADV_PATH+img))
    image = image.resize((IMG_HEIGHT,IMG_WIDTH))
    image = np.array(image)
    image = image/255.0
    adv_data.append(image)
    
adv_data = np.array(adv_data)

In [ ]:
plt.subplot(2,2,1)
plt.imshow(test_data[8])
plt.subplot(2,2,3)
plt.imshow(gen[8])
plt.subplot(2,2,2)
plt.imshow(test_data[18])
plt.subplot(2,2,4)
plt.imshow(gen[18])
plt.show()



def img_rmse(image1, image2):
  error = np.sqrt(np.sum((image1.astype('float')-image2.astype('float'))**2))
  #error /= image1.shape[0] * image1.shape[1]
  return error


rmse = []

for i in range(len(gen)):
    rmse.append(img_rmse(test_data[i], gen[i]))
    
plt.plot(rmse)
plt.show()

In [ ]:
adverse = ae.predict(adv_data)
armse = []

for i in range(len(adverse)):
    armse.append(img_rmse(adv_data[i], adverse[i]))
    
plt.plot(armse)
plt.show()

In [ ]:
plt.subplot(2,2,1)
plt.imshow(adv_data[8])
plt.subplot(2,2,3)
plt.imshow(adverse[8])
plt.subplot(2,2,2)
plt.imshow(adv_data[18])
plt.subplot(2,2,4)
plt.imshow(adverse[18])
plt.show()